Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from random import random, choice, randint
from copy import copy
from math import dist

In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets.toarray()

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [28]:
PROBLEM_SIZE = 5000
NUM_SETS = PROBLEM_SIZE
SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, .3)
print("Element at row=42 and column=42:", SETS[42, 42])

Element at row=42 and column=42: True


In [8]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness2(state):
    global COUNTER
    COUNTER += 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

fitness = fitness2

In [9]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

def tweak2(state):
    new_state = copy(state)
    n = randint(0, 2)
    indexes = [randint(0, PROBLEM_SIZE - 1) for _ in range(n)]
    for index in indexes:
        new_state[index] = not new_state[index]
    return new_state

def tweak3(state):
    new_state = copy(state)
    num_samples_insert = 10
    
    indexes = [randint(0, PROBLEM_SIZE-1) for _ in range(num_samples_insert)]
    distances = [dist(state, SETS[index]) for index in indexes]
    index = indexes[np.argmin(distances)]
    new_state[index] = not new_state[index]
    
    return new_state
    
tweak = tweak3

In [29]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#print(fitness(current_state))
COUNTER = 0
no_improvement = 0
best_fitness = (0, 0)

for step in range(10_000):
    new_state = tweak(current_state)
    no_improvement += 1
    new_fitness = fitness(new_state)
    if new_fitness > best_fitness:
        best_fitness = new_fitness
        current_state = new_state
        no_improvement = 0
        print(best_fitness)
    if no_improvement > 100:
        break

print(COUNTER)

(1446, -1)
(2491, -2)
(3230, -3)
(3736, -4)
(4099, -5)
(4376, -6)
(4582, -7)
(4694, -8)
(4779, -9)
(4850, -10)
(4899, -11)
(4925, -12)
(4955, -13)
(4966, -14)
(4974, -15)
(4987, -16)
(4992, -17)
(4994, -18)
(4998, -19)
(4999, -20)
(5000, -21)
(5000, -20)
213
